In [334]:
from plotnine import *
import pandas
from solarized import *
import tikzplotlib
pandas.options.display.max_rows=10

Index(['index', 'bytes_memory', 'cpu_percent', 'bytes_sent', 'bytes_recv',
       'wall_time', 'exitcode', 'db_time', 'rep', 'net', 'scale', 'wflow',
       'optimizer'],
      dtype='object')

In [ ]:
# 1.{5end_to_end} tpch 1,2,3,4,5 with scale 1, 10
# 2 {5module4} and 3. module4 | also {5module4mem} and {5module4net}
# 3.{5tpchmodin} tpch 1, 4, 5 with modin on scale 10
# 4.{5micro_scales} micros with scale 1, 10 | also {5micro_traffic}
# 5.{5micro_net} micro join, selection with net=wan and scale 1

In [ ]:
df=pandas.read_feather('specific-reports/5micro_net.feather')
df.columns

# CHECK

In [335]:
df['exitcode'].value_counts()

0    186285
1     21940
Name: exitcode, dtype: int64

In [336]:
df.query("exitcode == 1")['wflow'].unique()

array(['tpch1.py', 'q07.sql', 'q08.sql', 'q09.sql'], dtype=object)

# PREPROCESS

In [337]:
df['gb_memory'] = df['bytes_memory']/10**9
df.drop(columns=['bytes_memory'], inplace=True)

In [338]:
df['cpu'] = df['cpu_percent'].apply(lambda x: mean(x))
df.drop(columns=['cpu_percent'], inplace=True)

In [339]:
df['gb_net'] = df['bytes_recv']/10**9
df.drop(columns=['bytes_recv'], inplace=True)

In [341]:
df['procedural_time']=df['wall_time']-df['db_time']

In [384]:
micro = df[df['wflow'].isin(['micro_join.py','micro_sel.py','micro_proj.py','micro_max.py'])]

In [385]:
micro['rep'].value_counts()

2    106013
Name: rep, dtype: int64

In [386]:
# add id to interprocess measurements. Remove if snapshot_idx exists
micro['snapshot_idx'] = micro.groupby(factors)['wall_time'].rank(method='first').astype('int') # wall_time could be any column

<ipython-input-386-0d00b80f3679>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [387]:
interproc=micro[micro['rep']==2]
interproc=micro.drop(columns=['rep'])

In [399]:
overview = interproc.groupby(by=factors).first().reset_index().drop(columns=['exitcode', 'snapshot_idx'])
overview['scale'] = overview['scale'].astype('str')
overview.columns

Index(['wflow', 'optimizer', 'net', 'scale', 'index', 'bytes_sent',
       'wall_time', 'db_time', 'gb_memory', 'cpu', 'gb_net',
       'procedural_time'],
      dtype='object')

# PROCEDURAL AND DB TIME

In [ ]:
plot.save("specific-plots/5end_to_end.png")
matplot = plot.draw()
tikzplotlib.save('specific-plots/5end_to_end.tikz', figure=matplot)
display(plot)